# Unit 5 - Financial Planning

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools1 import MCSimulation
import json

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
# Fetch current BTC price
btc_data = requests.get(btc_url)

btc_data = btc_data.json()

# print(json.dumps(btc_data, indent = 4)) to identify the data dump structure

btc_price = btc_data['data']['1']['quotes']['CAD']['price']

# Fetch current ETH price
eth_data = requests.get(eth_url)

eth_data = eth_data.json()

# print(json.dumps(eth_data, indent = 4)) to identify the data dump structure

eth_price = eth_data['data']['1027']['quotes']['CAD']['price']

# Compute current value of my crpto
my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(api_key, secret_key, api_version = 'v2')

In [ ]:
# Format current date as ISO format
today = pd.Timestamp('2023-12-15', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG

tickers_df = api.get_bars(tickers, timeframe, start = today, end = today).df

tickers_df['symbol']

# Reorganize the DataFrame
# Separate ticker data

agg_df = tickers_df[tickers_df['symbol'] == "AGG"].drop('symbol',axis=1)
spy_df = tickers_df[tickers_df['symbol'] == "SPY"].drop('symbol',axis=1)   

# Concatenate the ticker DataFrames

new_tickers_df = pd.concat([agg_df, spy_df], axis = 1, keys=['AGG','SPY'])

# Preview DataFrame

new_tickers_df

In [ ]:
# Pick AGG and SPY close prices
agg_close_price = float(new_tickers_df['AGG']['close'])
spy_close_price = float(new_tickers_df['SPY']['close'])

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
my_spy_value = my_spy * spy_close_price
my_agg_value = my_agg * agg_close_price

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
shares_total = round(my_spy_value+my_agg_value,2)
crypto_total = round(my_eth_value+my_btc_value,2)

savings_dict = {'amount':[crypto_total,shares_total]}

# Create savings DataFrame
df_savings = pd.DataFrame(savings_dict, index = ['crypto','shares'])

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
df_savings.plot.pie(y = 'amount',title = 'Composition of Personal Savings')

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = df_savings['amount'].sum()

# Validate saving health
if total_savings > emergency_fund:
    print("Congratulations! You have enough money in your emergency fund.")
elif total_savings == emergency_fund:
    print("Congratulations! You have reached your financial saving's goal.")
else:
    print(f"You are ${emergency_fund - total_savings} away from reaching your financial saving's goal.")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2018-12-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-12-17', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
tickers_5yr_df = api.get_bars(tickers, timeframe, start = start_date, end = end_date).df

# Reorganize the DataFrame
# Separate ticker data
agg_5yr_df = tickers_5yr_df[tickers_5yr_df['symbol']=='AGG'].drop('symbol', axis=1)

spy_5yr_df = tickers_5yr_df[tickers_5yr_df['symbol']=='SPY'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
df_stock_data = pd.concat([agg_5yr_df, spy_5yr_df], axis=1, keys=['AGG','SPY'])

# Display sample data
df_stock_data

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30years = MCSimulation(
    portfolio_data = df_stock_data,
    weights=[0.40,0.60],
    num_simulation = 500,
    num_trading_days = 252*30)

In [ ]:
# Printing the simulation input data
MC_30years.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30years.calc_cumulative_return()

# Plot simulation outcomes
MC_30years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_30years.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
MC_30years_summary = MC_30years.summarize_cumulative_return()

# Print summary statistics
print(MC_30years_summary)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(MC_30years_summary[8]*initial_investment,2)
ci_upper = round(MC_30years_summary[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(MC_30years_summary[8]*initial_investment,2)
ci_upper = round(MC_30years_summary[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# 5 Years Simulation with Higher Stock to Bonds Ratio
MC_5years = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.25,0.75],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_5years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_5years.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
MC_5years_summary = MC_5years.summarize_cumulative_return()

# Print summary statistics
print(MC_5years_summary)

In [ ]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = initial_investment*MC_5years_summary[8]
ci_upper_five = initial_investment*MC_5years_summary[9]

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10years = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.25,0.75],
    num_simulation = 500,
    num_trading_days = 252*10
)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_10years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_10years.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
MC_10years_summary = MC_10years.summarize_cumulative_return()

# Print summary statistics
print(MC_10years_summary)

In [ ]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = initial_investment * MC_10years_summary[8]
ci_upper_ten = initial_investment * MC_10years_summary[9]

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")